# Notebook 02: Modeling + Evaluation + Hyperparameter Tuning

**Project:** Vehicle Sales & Market Insights  
**Dataset:** car_prices_cleaned.csv (from Notebook 01)  
**Purpose:** Build, evaluate, and optimize regression models for price prediction

## Objective
Develop production-ready price prediction models:
- Train/validation/test split (time-aware)
- Baseline models (Linear, Ridge, Random Forest)
- Advanced models (LightGBM, CatBoost, XGBoost)
- Hyperparameter optimization with Optuna
- Model evaluation and selection
- Save final model for deployment

## Step 1: Environment Setup
Load libraries and cleaned dataset.

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Modeling
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

# Advanced models
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb

# Hyperparameter tuning
import optuna

# Utilities
import pickle
import warnings
import os
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("Environment Setup Complete")
print(f"Python libraries loaded successfully")

# Load cleaned dataset
df = pd.read_csv('data/processed/car_prices_cleaned.csv')

print(f"\nDataset loaded: {df.shape}")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Target variable: sellingprice")
print(f"Features: {len(df.columns) - 1}")

Environment Setup Complete
Python libraries loaded successfully

Dataset loaded: (558825, 22)
Memory: 429.7 MB
Target variable: sellingprice
Features: 21


## Step 2: Feature Type Identification & Data Preparation

Identify feature types and prepare data for modeling.
Separate numeric features, categorical features, and non-modeling columns.

In [2]:
print("FEATURE TYPE IDENTIFICATION")
print("=" * 80)

# Display all columns
print(f"Total columns: {len(df.columns)}")
print(f"Column names: {list(df.columns)}\n")

# Define feature types
target = 'sellingprice'

numeric_features = ['year', 'condition', 'odometer', 'mmr', 'vehicle_age', 
                    'log_odometer', 'price_diff', 'mmr_ratio', 
                    'age_odo_interaction', 'has_date']

categorical_features = ['make', 'body', 'transmission', 'state', 'color', 
                       'interior', 'seller_grouped', 'model_grouped', 
                       'trim_grouped']

id_features = ['vin', 'saledate']

# Verify all columns accounted for
all_defined = set(numeric_features + categorical_features + id_features + [target])
all_actual = set(df.columns)
missing = all_actual - all_defined
extra = all_defined - all_actual

print("Feature Categories:")
print(f"\nNumeric features ({len(numeric_features)}):")
for feat in numeric_features:
    print(f"  - {feat}")

print(f"\nCategorical features ({len(categorical_features)}):")
for feat in categorical_features:
    print(f"  - {feat} ({df[feat].nunique()} unique)")

print(f"\nID/Reference features ({len(id_features)}):")
for feat in id_features:
    print(f"  - {feat}")

print(f"\nTarget: {target}")

if missing:
    print(f"\nWarning: Columns not categorized: {missing}")
if extra:
    print(f"\nWarning: Defined but not in data: {extra}")

# Check for missing values
print("\n" + "-" * 80)
print("Missing Values Check:")
missing_counts = df[numeric_features + categorical_features].isnull().sum()
if missing_counts.sum() == 0:
    print("All modeling features are complete!")
else:
    print(missing_counts[missing_counts > 0])

# Target variable statistics
print("\n" + "-" * 80)
print("Target Variable Statistics:")
print(f"Mean: ${df[target].mean():,.2f}")
print(f"Median: ${df[target].median():,.2f}")
print(f"Std: ${df[target].std():,.2f}")
print(f"Min: ${df[target].min():,.2f}")
print(f"Max: ${df[target].max():,.2f}")
print(f"Missing: {df[target].isnull().sum()}")

FEATURE TYPE IDENTIFICATION
Total columns: 22
Column names: ['year', 'make', 'body', 'transmission', 'vin', 'state', 'condition', 'odometer', 'color', 'interior', 'mmr', 'sellingprice', 'saledate', 'has_date', 'vehicle_age', 'log_odometer', 'price_diff', 'mmr_ratio', 'age_odo_interaction', 'seller_grouped', 'model_grouped', 'trim_grouped']

Feature Categories:

Numeric features (10):
  - year
  - condition
  - odometer
  - mmr
  - vehicle_age
  - log_odometer
  - price_diff
  - mmr_ratio
  - age_odo_interaction
  - has_date

Categorical features (9):
  - make (67 unique)
  - body (46 unique)
  - transmission (2 unique)
  - state (64 unique)
  - color (46 unique)
  - interior (17 unique)
  - seller_grouped (101 unique)
  - model_grouped (201 unique)
  - trim_grouped (101 unique)

ID/Reference features (2):
  - vin
  - saledate

Target: sellingprice

--------------------------------------------------------------------------------
Missing Values Check:
All modeling features are complete!


## Step 3: Encode Categorical Features

Apply label encoding to categorical features for tree-based models.
These models handle encoded categories naturally without one-hot encoding.

In [3]:
print("CATEGORICAL FEATURE ENCODING")
print("=" * 80)

# Create copy for encoding
df_encoded = df.copy()

# Label encode categorical features
print("Encoding categorical features:\n")
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"{col:20s}: {df[col].nunique():3d} categories → encoded as 0-{df_encoded[col].max()}")

# Save encoders for later use
encoders_path = 'models/preprocessing/label_encoders.pkl'
os.makedirs('models/preprocessing', exist_ok=True)
with open(encoders_path, 'wb') as f:
    pickle.dump(label_encoders, f)

print(f"\nLabel encoders saved: {encoders_path}")

# Prepare feature matrix and target
X = df_encoded[numeric_features + categorical_features]
y = df_encoded[target]

print("\n" + "-" * 80)
print("Feature Matrix Prepared:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
print(f"  Numeric: {numeric_features}")
print(f"  Categorical (encoded): {categorical_features}")

# Verify no missing values
print(f"\nMissing values in X: {X.isnull().sum().sum()}")
print(f"Missing values in y: {y.isnull().sum()}")

CATEGORICAL FEATURE ENCODING
Encoding categorical features:

make                :  67 categories → encoded as 0-66
body                :  46 categories → encoded as 0-45
transmission        :   2 categories → encoded as 0-1
state               :  64 categories → encoded as 0-63
color               :  46 categories → encoded as 0-45
interior            :  17 categories → encoded as 0-16
seller_grouped      : 101 categories → encoded as 0-100
model_grouped       : 201 categories → encoded as 0-200
trim_grouped        : 101 categories → encoded as 0-100

Label encoders saved: models/preprocessing/label_encoders.pkl

--------------------------------------------------------------------------------
Feature Matrix Prepared:
X shape: (558825, 19)
y shape: (558825,)

Feature columns (19):
  Numeric: ['year', 'condition', 'odometer', 'mmr', 'vehicle_age', 'log_odometer', 'price_diff', 'mmr_ratio', 'age_odo_interaction', 'has_date']
  Categorical (encoded): ['make', 'body', 'transmission', 'stat

## Step 4: Train/Validation/Test Split

Create train/validation/test splits with stratified sampling.
Use 70% train, 15% validation, 15% test to ensure sufficient data for all sets.

In [4]:
print("TRAIN/VALIDATION/TEST SPLIT")
print("=" * 80)

# Set random seed for reproducibility
RANDOM_STATE = 42

# Split: 70% train, 15% validation, 15% test
# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_STATE
)

# Second split: 50% of temp for validation, 50% for test (15% each of total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=RANDOM_STATE
)

print("Split Sizes:")
print(f"  Training set:   {len(X_train):,} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Validation set: {len(X_val):,} ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Test set:       {len(X_test):,} ({len(X_test)/len(X)*100:.1f}%)")
print(f"  Total:          {len(X):,}")

# Verify target distribution across splits
print("\n" + "-" * 80)
print("Target Distribution Across Splits:")
print(f"  Train   - Mean: ${y_train.mean():,.2f}, Std: ${y_train.std():,.2f}")
print(f"  Val     - Mean: ${y_val.mean():,.2f}, Std: ${y_val.std():,.2f}")
print(f"  Test    - Mean: ${y_test.mean():,.2f}, Std: ${y_test.std():,.2f}")
print(f"  Overall - Mean: ${y.mean():,.2f}, Std: ${y.std():,.2f}")

# Check feature distributions
print("\n" + "-" * 80)
print("Feature Statistics (Training Set):")
print(f"  Numeric features mean odometer: {X_train['odometer'].mean():,.0f}")
print(f"  Numeric features mean vehicle_age: {X_train['vehicle_age'].mean():.1f}")
print(f"  Numeric features mean mmr: ${X_train['mmr'].mean():,.2f}")

print("\n" + "=" * 80)
print("Data split complete and ready for modeling!")

TRAIN/VALIDATION/TEST SPLIT
Split Sizes:
  Training set:   391,177 (70.0%)
  Validation set: 83,824 (15.0%)
  Test set:       83,824 (15.0%)
  Total:          558,825

--------------------------------------------------------------------------------
Target Distribution Across Splits:
  Train   - Mean: $13,605.71, Std: $9,756.17
  Val     - Mean: $13,613.10, Std: $9,681.98
  Test    - Mean: $13,635.96, Std: $9,785.69
  Overall - Mean: $13,611.36, Std: $9,749.50

--------------------------------------------------------------------------------
Feature Statistics (Training Set):
  Numeric features mean odometer: 68,311
  Numeric features mean vehicle_age: 5.0
  Numeric features mean mmr: $13,764.58

Data split complete and ready for modeling!


## Step 5: Train Baseline Models

Establish performance baselines with simple models:
- Linear Regression (baseline)
- Ridge Regression (regularized linear)
- Random Forest (tree-based ensemble)

Evaluate using MAE, RMSE, R², and MAPE metrics.

In [5]:
print("BASELINE MODEL TRAINING")
print("=" * 80)

# Define evaluation function
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    
    return {
        'Model': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'MAPE': mape
    }

results = []

# 1. Linear Regression
print("1. Training Linear Regression...")
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)
results.append(evaluate_model(y_val, y_pred_lr, 'Linear Regression'))
print("   ✓ Complete")

# 2. Ridge Regression
print("2. Training Ridge Regression...")
ridge = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_val)
results.append(evaluate_model(y_val, y_pred_ridge, 'Ridge'))
print("   ✓ Complete")

# 3. Random Forest
print("3. Training Random Forest...")
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=20,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
results.append(evaluate_model(y_val, y_pred_rf, 'Random Forest'))
print("   ✓ Complete")

# Display results
print("\n" + "=" * 80)
print("BASELINE MODEL RESULTS (Validation Set)")
print("=" * 80)
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

print("\n" + "-" * 80)
print("Interpretation:")
print(f"  Best MAE:  {results_df.loc[results_df['MAE'].idxmin(), 'Model']} (${results_df['MAE'].min():,.2f})")
print(f"  Best RMSE: {results_df.loc[results_df['RMSE'].idxmin(), 'Model']} (${results_df['RMSE'].min():,.2f})")
print(f"  Best R²:   {results_df.loc[results_df['R2'].idxmax(), 'Model']} ({results_df['R2'].max():.4f})")
print(f"  Best MAPE: {results_df.loc[results_df['MAPE'].idxmin(), 'Model']} ({results_df['MAPE'].min():.2f}%)")

BASELINE MODEL TRAINING
1. Training Linear Regression...
   ✓ Complete
2. Training Ridge Regression...
   ✓ Complete
3. Training Random Forest...
   ✓ Complete

BASELINE MODEL RESULTS (Validation Set)
            Model          MAE         RMSE       R2         MAPE
Linear Regression 1.458705e-11 1.983436e-11 1.000000 2.559347e-13
            Ridge 1.006473e-09 1.631534e-09 1.000000 1.458081e-11
    Random Forest 1.386135e+01 1.631839e+02 0.999716 1.391432e-01

--------------------------------------------------------------------------------
Interpretation:
  Best MAE:  Linear Regression ($0.00)
  Best RMSE: Linear Regression ($0.00)
  Best R²:   Linear Regression (1.0000)
  Best MAPE: Linear Regression (0.00%)


## Step 6: Fix Data Leakage

Remove features derived from the target variable (price_diff, mmr_ratio).
These cause data leakage and artificial perfect predictions.
Retrain baseline models with clean features.

In [6]:
print("FIXING DATA LEAKAGE")
print("=" * 80)

print("Identifying leakage features:")
print("  - price_diff = sellingprice - mmr (LEAKAGE)")
print("  - mmr_ratio = sellingprice / mmr (LEAKAGE)")
print("\nThese features are derived FROM the target and must be removed.")

# Remove leakage features
leakage_features = ['price_diff', 'mmr_ratio']
numeric_features_clean = [f for f in numeric_features if f not in leakage_features]

print(f"\nNumeric features before: {len(numeric_features)}")
print(f"Numeric features after:  {len(numeric_features_clean)}")

# Recreate feature matrix without leakage
X_clean = df_encoded[numeric_features_clean + categorical_features]

print(f"\nNew feature matrix: {X_clean.shape}")
print(f"Features removed: {leakage_features}")
print(f"Remaining features ({len(X_clean.columns)}): {list(X_clean.columns)}")

# Re-split data
X_train, X_temp, y_train, y_temp = train_test_split(
    X_clean, y, test_size=0.30, random_state=RANDOM_STATE
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=RANDOM_STATE
)

print("\n" + "-" * 80)
print("Data re-split with clean features:")
print(f"  Training:   {X_train.shape}")
print(f"  Validation: {X_val.shape}")
print(f"  Test:       {X_test.shape}")

FIXING DATA LEAKAGE
Identifying leakage features:
  - price_diff = sellingprice - mmr (LEAKAGE)
  - mmr_ratio = sellingprice / mmr (LEAKAGE)

These features are derived FROM the target and must be removed.

Numeric features before: 10
Numeric features after:  8

New feature matrix: (558825, 17)
Features removed: ['price_diff', 'mmr_ratio']
Remaining features (17): ['year', 'condition', 'odometer', 'mmr', 'vehicle_age', 'log_odometer', 'age_odo_interaction', 'has_date', 'make', 'body', 'transmission', 'state', 'color', 'interior', 'seller_grouped', 'model_grouped', 'trim_grouped']

--------------------------------------------------------------------------------
Data re-split with clean features:
  Training:   (391177, 17)
  Validation: (83824, 17)
  Test:       (83824, 17)


## Step 7: Retrain Baseline Models with Clean Features

Train baseline models again without data leakage.
This will show realistic model performance for price prediction.

In [7]:
print("BASELINE MODEL TRAINING (CLEAN FEATURES)")
print("=" * 80)

results_clean = []

# 1. Linear Regression
print("1. Training Linear Regression...")
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)
results_clean.append(evaluate_model(y_val, y_pred_lr, 'Linear Regression'))
print("   ✓ Complete")

# 2. Ridge Regression
print("2. Training Ridge Regression...")
ridge = Ridge(alpha=10.0, random_state=RANDOM_STATE)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_val)
results_clean.append(evaluate_model(y_val, y_pred_ridge, 'Ridge'))
print("   ✓ Complete")

# 3. Random Forest
print("3. Training Random Forest...")
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=20,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=0
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
results_clean.append(evaluate_model(y_val, y_pred_rf, 'Random Forest'))
print("   ✓ Complete")

# Display results
print("\n" + "=" * 80)
print("BASELINE MODEL RESULTS (Validation Set - Clean Features)")
print("=" * 80)
results_df = pd.DataFrame(results_clean)
print(results_df.to_string(index=False))

print("\n" + "-" * 80)
print("Best Performers:")
print(f"  Best MAE:  {results_df.loc[results_df['MAE'].idxmin(), 'Model']} (${results_df['MAE'].min():,.2f})")
print(f"  Best RMSE: {results_df.loc[results_df['RMSE'].idxmin(), 'Model']} (${results_df['RMSE'].min():,.2f})")
print(f"  Best R²:   {results_df.loc[results_df['R2'].idxmax(), 'Model']} ({results_df['R2'].max():.4f})")
print(f"  Best MAPE: {results_df.loc[results_df['MAPE'].idxmin(), 'Model']} ({results_df['MAPE'].min():.2f}%)")

print("\n" + "-" * 80)
print("Insights:")
print(f"  Random Forest shows strong performance with R² = {results_df.loc[results_df['Model']=='Random Forest', 'R2'].values[0]:.4f}")
print(f"  Linear models have lower R² due to non-linear relationships in data")
print(f"  Tree-based models better capture complex feature interactions")

BASELINE MODEL TRAINING (CLEAN FEATURES)
1. Training Linear Regression...
   ✓ Complete
2. Training Ridge Regression...
   ✓ Complete
3. Training Random Forest...
   ✓ Complete

BASELINE MODEL RESULTS (Validation Set - Clean Features)
            Model         MAE        RMSE       R2      MAPE
Linear Regression 1053.310882 1643.373110 0.971190 15.701325
            Ridge 1053.310476 1643.372976 0.971190 15.701273
    Random Forest  932.418677 1477.742741 0.976704 13.446442

--------------------------------------------------------------------------------
Best Performers:
  Best MAE:  Random Forest ($932.42)
  Best RMSE: Random Forest ($1,477.74)
  Best R²:   Random Forest (0.9767)
  Best MAPE: Random Forest (13.45%)

--------------------------------------------------------------------------------
Insights:
  Random Forest shows strong performance with R² = 0.9767
  Linear models have lower R² due to non-linear relationships in data
  Tree-based models better capture complex feature int

## Step 8: Train Advanced Gradient Boosting Models

Train state-of-the-art gradient boosting models:
- LightGBM (fast, efficient)
- XGBoost (robust, widely used)
- CatBoost (handles categoricals natively)

Compare against Random Forest baseline.

In [8]:
print("ADVANCED MODEL TRAINING")
print("=" * 80)

# 1. LightGBM
print("1. Training LightGBM...")
lgb_model = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=15,
    num_leaves=100,
    min_child_samples=20,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_val)
results_clean.append(evaluate_model(y_val, y_pred_lgb, 'LightGBM'))
print("   ✓ Complete")

# 2. XGBoost
print("2. Training XGBoost...")
xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=10,
    min_child_weight=5,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_val)
results_clean.append(evaluate_model(y_val, y_pred_xgb, 'XGBoost'))
print("   ✓ Complete")

# 3. CatBoost
print("3. Training CatBoost...")
cat_model = CatBoostRegressor(
    iterations=200,
    learning_rate=0.05,
    depth=10,
    random_state=RANDOM_STATE,
    verbose=0
)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_val)
results_clean.append(evaluate_model(y_val, y_pred_cat, 'CatBoost'))
print("   ✓ Complete")

# Display all results
print("\n" + "=" * 80)
print("ALL MODEL RESULTS (Validation Set)")
print("=" * 80)
results_df = pd.DataFrame(results_clean)
results_df = results_df.sort_values('MAE')
print(results_df.to_string(index=False))

print("\n" + "-" * 80)
print("Best Model by Metric:")
print(f"  Lowest MAE:  {results_df.iloc[0]['Model']} (${results_df.iloc[0]['MAE']:,.2f})")
print(f"  Lowest RMSE: {results_df.loc[results_df['RMSE'].idxmin(), 'Model']} (${results_df['RMSE'].min():,.2f})")
print(f"  Highest R²:  {results_df.loc[results_df['R2'].idxmax(), 'Model']} ({results_df['R2'].max():.4f})")
print(f"  Lowest MAPE: {results_df.loc[results_df['MAPE'].idxmin(), 'Model']} ({results_df['MAPE'].min():.2f}%)")

print("\n" + "=" * 80)
print(f"Champion Model: {results_df.iloc[0]['Model']}")
print(f"Performance: ${results_df.iloc[0]['MAE']:,.2f} MAE, {results_df.iloc[0]['R2']:.4f} R²")

ADVANCED MODEL TRAINING
1. Training LightGBM...
   ✓ Complete
2. Training XGBoost...
   ✓ Complete
3. Training CatBoost...
   ✓ Complete

ALL MODEL RESULTS (Validation Set)
            Model         MAE        RMSE       R2      MAPE
          XGBoost  905.979434 1567.303833 0.973795 12.733463
         LightGBM  906.705690 1510.702516 0.975654 13.098833
    Random Forest  932.418677 1477.742741 0.976704 13.446442
         CatBoost  952.641952 1643.748533 0.971176 13.842275
            Ridge 1053.310476 1643.372976 0.971190 15.701273
Linear Regression 1053.310882 1643.373110 0.971190 15.701325

--------------------------------------------------------------------------------
Best Model by Metric:
  Lowest MAE:  XGBoost ($905.98)
  Lowest RMSE: Random Forest ($1,477.74)
  Highest R²:  Random Forest (0.9767)
  Lowest MAPE: XGBoost (12.73%)

Champion Model: XGBoost
Performance: $905.98 MAE, 0.9738 R²


## Step 9: Hyperparameter Optimization with Optuna

Use Optuna to find optimal hyperparameters for XGBoost.
Optimize for MAE (Mean Absolute Error) to minimize prediction error.
Run 50 trials to balance search time and performance.

In [9]:
print("HYPERPARAMETER OPTIMIZATION WITH OPTUNA")
print("=" * 80)

# Define objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 6, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbosity': 0
    }
    
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    
    return mae

print("Starting Optuna optimization...")
print(f"Metric to optimize: MAE (Mean Absolute Error)")
print(f"Number of trials: 50")
print(f"This may take 5-10 minutes...\n")

# Create study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n" + "=" * 80)
print("OPTIMIZATION RESULTS")
print("=" * 80)
print(f"Best MAE: ${study.best_value:,.2f}")
print(f"\nBest Hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Train final model with best parameters
print("\n" + "-" * 80)
print("Training final optimized model...")
best_params = study.best_params
best_params.update({'random_state': RANDOM_STATE, 'n_jobs': -1, 'verbosity': 0})

xgb_optimized = xgb.XGBRegressor(**best_params)
xgb_optimized.fit(X_train, y_train)
y_pred_optimized = xgb_optimized.predict(X_val)

print("✓ Complete")

# Evaluate optimized model
result_optimized = evaluate_model(y_val, y_pred_optimized, 'XGBoost (Optimized)')
print("\n" + "=" * 80)
print("OPTIMIZED MODEL PERFORMANCE")
print("=" * 80)
print(f"MAE:  ${result_optimized['MAE']:,.2f}")
print(f"RMSE: ${result_optimized['RMSE']:,.2f}")
print(f"R²:   {result_optimized['R2']:.4f}")
print(f"MAPE: {result_optimized['MAPE']:.2f}%")

# Compare with baseline XGBoost
baseline_mae = results_df[results_df['Model'] == 'XGBoost']['MAE'].values[0]
improvement = baseline_mae - result_optimized['MAE']
print(f"\nImprovement over baseline XGBoost: ${improvement:,.2f} ({improvement/baseline_mae*100:.2f}%)")

HYPERPARAMETER OPTIMIZATION WITH OPTUNA
Starting Optuna optimization...
Metric to optimize: MAE (Mean Absolute Error)
Number of trials: 50
This may take 5-10 minutes...



  0%|          | 0/50 [00:00<?, ?it/s]


OPTIMIZATION RESULTS
Best MAE: $886.53

Best Hyperparameters:
  n_estimators: 448
  learning_rate: 0.0452419039553789
  max_depth: 12
  min_child_weight: 6
  subsample: 0.9393774616218383
  colsample_bytree: 0.7938614515522494
  gamma: 1.5790987446846159

--------------------------------------------------------------------------------
Training final optimized model...
✓ Complete

OPTIMIZED MODEL PERFORMANCE
MAE:  $886.53
RMSE: $1,527.48
R²:   0.9751
MAPE: 12.40%

Improvement over baseline XGBoost: $19.45 (2.15%)


## Step 10: Final Model Evaluation on Test Set

Evaluate the optimized XGBoost model on the held-out test set.
This confirms the model generalizes well to unseen data.

In [10]:
print("FINAL MODEL EVALUATION ON TEST SET")
print("=" * 80)

# Predict on test set
y_pred_test = xgb_optimized.predict(X_test)

# Calculate metrics
test_results = evaluate_model(y_test, y_pred_test, 'XGBoost (Optimized)')

print("Test Set Performance:")
print(f"  MAE:  ${test_results['MAE']:,.2f}")
print(f"  RMSE: ${test_results['RMSE']:,.2f}")
print(f"  R²:   {test_results['R2']:.4f}")
print(f"  MAPE: {test_results['MAPE']:.2f}%")

# Compare validation vs test
print("\n" + "-" * 80)
print("Validation vs Test Comparison:")
print(f"  MAE:  ${result_optimized['MAE']:,.2f} (val) vs ${test_results['MAE']:,.2f} (test)")
print(f"  RMSE: ${result_optimized['RMSE']:,.2f} (val) vs ${test_results['RMSE']:,.2f} (test)")
print(f"  R²:   {result_optimized['R2']:.4f} (val) vs {test_results['R2']:.4f} (test)")
print(f"  MAPE: {result_optimized['MAPE']:.2f}% (val) vs {test_results['MAPE']:.2f}% (test)")

# Check for overfitting
mae_diff = abs(test_results['MAE'] - result_optimized['MAE'])
r2_diff = abs(test_results['R2'] - result_optimized['R2'])

print("\n" + "-" * 80)
print("Generalization Check:")
if mae_diff < 50 and r2_diff < 0.01:
    print("  ✓ Model generalizes well - minimal difference between validation and test")
elif mae_diff < 100 and r2_diff < 0.02:
    print("  ✓ Model generalizes reasonably - acceptable difference")
else:
    print("  ⚠ Potential overfitting - significant performance drop on test set")

print(f"  MAE difference: ${mae_diff:,.2f}")
print(f"  R² difference: {r2_diff:.4f}")

# Prediction examples
print("\n" + "=" * 80)
print("Sample Predictions (First 10 test samples):")
print("=" * 80)
sample_results = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted': y_pred_test[:10],
    'Error': y_test.values[:10] - y_pred_test[:10],
    'Error_%': ((y_test.values[:10] - y_pred_test[:10]) / y_test.values[:10] * 100)
})
print(sample_results.to_string(index=False))

print(f"\n  Mean absolute error in sample: ${abs(sample_results['Error']).mean():,.2f}")

FINAL MODEL EVALUATION ON TEST SET
Test Set Performance:
  MAE:  $887.48
  RMSE: $1,745.86
  R²:   0.9682
  MAPE: 12.30%

--------------------------------------------------------------------------------
Validation vs Test Comparison:
  MAE:  $886.53 (val) vs $887.48 (test)
  RMSE: $1,527.48 (val) vs $1,745.86 (test)
  R²:   0.9751 (val) vs 0.9682 (test)
  MAPE: 12.40% (val) vs 12.30% (test)

--------------------------------------------------------------------------------
Generalization Check:
  ✓ Model generalizes well - minimal difference between validation and test
  MAE difference: $0.96
  R² difference: 0.0069

Sample Predictions (First 10 test samples):
 Actual    Predicted       Error   Error_%
 7000.0  6743.246094  256.753906  3.667913
 8500.0  8949.000977 -449.000977 -5.282364
 6900.0  5942.892578  957.107422 13.871122
 1550.0   826.626648  723.373352 46.669249
13700.0 13729.688477  -29.688477 -0.216704
32200.0 30120.544922 2079.455078  6.457935
10500.0 11053.075195 -553.075195

## Step 11: Save Final Model and Artifacts

Save the trained model, label encoders, and metadata for deployment.
Package everything needed for production inference.

In [11]:
print("SAVING FINAL MODEL AND ARTIFACTS")
print("=" * 80)

# Create directories
os.makedirs('models/final', exist_ok=True)

# 1. Save the optimized model
model_path = 'models/final/xgboost_optimized.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(xgb_optimized, f)
print(f"✓ Model saved: {model_path}")

# 2. Save model metadata
metadata = {
    'model_type': 'XGBoost Regressor',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'features': list(X_train.columns),
    'numeric_features': numeric_features_clean,
    'categorical_features': categorical_features,
    'n_train_samples': len(X_train),
    'n_val_samples': len(X_val),
    'n_test_samples': len(X_test),
    'hyperparameters': study.best_params,
    'performance': {
        'validation_mae': result_optimized['MAE'],
        'validation_rmse': result_optimized['RMSE'],
        'validation_r2': result_optimized['R2'],
        'validation_mape': result_optimized['MAPE'],
        'test_mae': test_results['MAE'],
        'test_rmse': test_results['RMSE'],
        'test_r2': test_results['R2'],
        'test_mape': test_results['MAPE']
    }
}

metadata_path = 'models/final/model_metadata.pkl'
with open(metadata_path, 'wb') as f:
    pickle.dump(metadata, f)
print(f"✓ Metadata saved: {metadata_path}")

# 3. Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_optimized.feature_importances_
}).sort_values('importance', ascending=False)

importance_path = 'models/final/feature_importance.csv'
feature_importance.to_csv(importance_path, index=False)
print(f"✓ Feature importance saved: {importance_path}")

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

# 4. Save complete training configuration
config = {
    'random_state': RANDOM_STATE,
    'train_split': 0.70,
    'val_split': 0.15,
    'test_split': 0.15,
    'removed_features': ['price_diff', 'mmr_ratio'],
    'target': 'sellingprice'
}

config_path = 'models/final/training_config.pkl'
with open(config_path, 'wb') as f:
    pickle.dump(config, f)
print(f"\n✓ Training config saved: {config_path}")

print("\n" + "=" * 80)
print("MODEL DEPLOYMENT PACKAGE READY")
print("=" * 80)
print(f"Location: models/final/")
print(f"Files created:")
print(f"  1. xgboost_optimized.pkl (trained model)")
print(f"  2. model_metadata.pkl (performance metrics & config)")
print(f"  3. feature_importance.csv (feature rankings)")
print(f"  4. training_config.pkl (training parameters)")
print(f"  5. label_encoders.pkl (from Step 3, in models/preprocessing/)")

print("\n" + "=" * 80)
print("NOTEBOOK 02 COMPLETE!")
print("=" * 80)
print(f"Final Model: XGBoost (Optimized)")
print(f"Test Performance: ${test_results['MAE']:,.2f} MAE, {test_results['R2']:.4f} R², {test_results['MAPE']:.2f}% MAPE")
print(f"\nNext: Notebook 03 - Explainability + Fairness + Diagnostics")

SAVING FINAL MODEL AND ARTIFACTS
✓ Model saved: models/final/xgboost_optimized.pkl
✓ Metadata saved: models/final/model_metadata.pkl
✓ Feature importance saved: models/final/feature_importance.csv

Top 10 Most Important Features:
            feature  importance
                mmr    0.616879
               body    0.089662
age_odo_interaction    0.084514
               make    0.058557
      model_grouped    0.030040
               year    0.028413
           odometer    0.027547
          condition    0.012360
       trim_grouped    0.012269
       transmission    0.009032

✓ Training config saved: models/final/training_config.pkl

MODEL DEPLOYMENT PACKAGE READY
Location: models/final/
Files created:
  1. xgboost_optimized.pkl (trained model)
  2. model_metadata.pkl (performance metrics & config)
  3. feature_importance.csv (feature rankings)
  4. training_config.pkl (training parameters)
  5. label_encoders.pkl (from Step 3, in models/preprocessing/)

NOTEBOOK 02 COMPLETE!
Final Mod